In [ ]:
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
from skimage.transform import resize
from skimage.util import img_as_ubyte
import io
import urllib
from PIL import Image
import pandas as pd

In [ ]:
df = pd.read_csv("/content/iphone_sim.csv", encoding='utf-8')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52150 entries, 0 to 52149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nom amazon    52150 non-null  object 
 1   link_amazon   52150 non-null  object 
 2   Nom express   52150 non-null  object 
 3   link_express  52150 non-null  object 
 4   sim           52150 non-null  float64
dtypes: float64(1), object(4)
memory usage: 2.0+ MB


In [ ]:
df_filtered = df[df['sim'] > 0.8]

In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1782 entries, 85 to 52082
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nom amazon    1782 non-null   object 
 1   link_amazon   1782 non-null   object 
 2   Nom express   1782 non-null   object 
 3   link_express  1782 non-null   object 
 4   sim           1782 non-null   float64
dtypes: float64(1), object(4)
memory usage: 83.5+ KB


In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(text1, text2):
    # Séparer les mots
    words_s1 = text1.split()
    words_s2 = text2.split()

    # Choisir le nombre minimum de mots entre les deux chaînes
    num_words = min(len(words_s1), len(words_s2))
    # Tronquer les deux chaînes
    text1 = ' '.join(words_s1[:num_words])
    text2 = ' '.join(words_s2[:num_words])

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])

    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]

    return similarity

In [ ]:
df_filtered['Nom amazon'][85]

'Apple iPhone 11, 64Go, Noir (Reconditionné)'

In [ ]:
df_filtered['Nom express'][85]

'Apple iPhone 11 64GB 128GB 256GB 6.1" Original Liquid Retina IPS LCD FACE ID A13 Genuine iPhone11 Unlocked 4G LTE Mobile Phone'

In [ ]:
s1 = "Apple iPhone 11 64GB 128GB 256GB 6.1 Original Liquid Retina IPS LCD FACE ID A13 Genuine iPhone11 Unlocked 4G LTE Mobile Phone"
s2 = "Apple iPhone 11, 64Go, Noir (Reconditionné)"

# Séparer les mots
words_s1 = s1.split()
words_s2 = s2.split()

# Choisir le nombre minimum de mots entre les deux chaînes
num_words = min(len(words_s1), len(words_s2))

# Tronquer les deux chaînes
s1_truncated = ' '.join(words_s1[:num_words])
s2_truncated = ' '.join(words_s2[:num_words])

print(s1_truncated)
print(s2_truncated)


Apple iPhone 11 64GB 128GB 256GB
Apple iPhone 11, 64Go, Noir (Reconditionné)


In [ ]:
calculate_cosine_similarity(df_filtered['Nom amazon'][85], df_filtered['Nom express'][85])

0.33609692727625745

In [ ]:
calculate_cosine_similarity("hella abc", "hello abc")

0.33609692727625756

In [ ]:
df_filtered['cosin_sim'] = df_filtered.apply(lambda row: calculate_cosine_similarity(row['Nom amazon'], row['Nom express']), axis=1)

<ipython-input-42-ef5be1c405ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cosin_sim'] = df_filtered.apply(lambda row: calculate_cosine_similarity(row['Nom amazon'], row['Nom express']), axis=1)


In [ ]:
df_filtered.head()

,Nom amazon,link_amazon,Nom express,link_express,sim,cosin_sim
85,"Apple iPhone 11, 64Go, Noir (Reconditionné)",https://m.media-amazon.com/images/I/61cnmobWl4...,"Apple iPhone 11 64GB 128GB 256GB 6.1"" Original...",https://ae01.alicdn.com/kf/Sfae1a9f67c0b4929b7...,0.860641,0.336097
280,"Apple iPhone 12 Pro, 128GB, Graphite - (Recond...",https://m.media-amazon.com/images/I/61DreMaVpl...,Apple iPhone 13 Pro 128GB 256GB ROM A15 Face I...,https://ae01.alicdn.com/kf/S721d2644f7ed45c38e...,0.810496,0.368023
455,"Apple iPhone 12 Pro Max, 128GB, Graphite - (Re...",https://m.media-amazon.com/images/I/71XXJC7V8t...,Apple iPhone 13 Pro 128GB 256GB ROM A15 Face I...,https://ae01.alicdn.com/kf/S721d2644f7ed45c38e...,0.810690,0.311257
725,Apple iPhone 11 128Go - Noir - Débloqué (Recon...,https://m.media-amazon.com/images/I/710eZ8NwLo...,Unlocked Original iPhone SE 2020 SE2 A13 3G RA...,https://ae01.alicdn.com/kf/S0574a865c9c34603b3...,0.812327,0.068050
785,Apple iPhone 11 128Go - Noir - Débloqué (Recon...,https://m.media-amazon.com/images/I/710eZ8NwLo...,"Apple iPhone 11 64GB 128GB 256GB 6.1"" Original...",https://ae01.alicdn.com/kf/Sfae1a9f67c0b4929b7...,0.864854,0.253209


In [ ]:
result_df = df_filtered[df_filtered['cosin_sim'] > 0.3]

In [ ]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 85 to 27321
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nom amazon    12 non-null     object 
 1   link_amazon   12 non-null     object 
 2   Nom express   12 non-null     object 
 3   link_express  12 non-null     object 
 4   sim           12 non-null     float64
 5   cosin_sim     12 non-null     float64
dtypes: float64(2), object(4)
memory usage: 672.0+ bytes


In [ ]:
for i, row in result_df.iterrows():
